<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/Nadav/TFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install pytorch-forecasting pytorch-lightning

In [2]:
from google.colab import drive
drive.flush_and_unmount()

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
import torch
from datetime import datetime

In [5]:
cleaned_data = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/cleaned_data.csv')

<ipython-input-5-20e4947fae02>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_data = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/cleaned_data.csv')


In [6]:
desc_map = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/desc_encoding_map.csv')
cat_map = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/category_mapping.csv')

In [7]:
# עותק מהנתונים המקוריים
df = cleaned_data.copy()

In [8]:
from pytorch_forecasting import TimeSeriesDataSet
import pandas as pd

def build_tft_datasets_from_raw(
    df,
    desc_map,
    cat_map,
    target_col="כמות",
    time_idx_col="order",
    group_col="clean_desc_str",
    date_col="Date",
    cutoff_date="2024-01-01",
    encoder_length=30,
    prediction_length=7,
):
    # שלב 1 – מיזוג מיפויים
    desc_map = desc_map.rename(columns={
        "code": "clean_desc_encoded",
        "Unnamed: 0": "clean_desc_str"
    })
    cat_map = cat_map.rename(columns={
        "קוד": "category_encoded",
        "קטגוריה": "category_str"
    })
    df = df.merge(desc_map, on="clean_desc_encoded", how="left")
    df = df.merge(cat_map, on="category_encoded", how="left")

    # שלב 2 – ניקוי והמרת טיפוסים
    df[date_col]           = pd.to_datetime(df[date_col])
    df["clean_desc_str"]   = df["clean_desc_str"].astype(str).fillna("Unknown")
    df["category_str"]     = df["category_str"].astype(str).fillna("Unknown")
    df["portion_type"]     = df["portion_type"].astype(str).fillna("Unknown")
    df[target_col]         = pd.to_numeric(df[target_col], errors="coerce")
    df["time_idx"]         = df[time_idx_col]

    # **_drop all rows with the problematic code 54 right here**
    df = df[df["clean_desc_encoded"] != 54].copy()

    # שלב 3 – פיצול דאטה
    train_df = df[df[date_col] < cutoff_date].copy()
    val_df   = df[df[date_col] >= cutoff_date].copy()

    # שלב 4 – יצירת TimeSeriesDataSet לאימון
    known_reals         = ["time_idx","Month","Day","Day_Name","Is_Weekend",
                           "is_christian_holiday","is_jewish_holiday","is_near_jewish_holiday",
                           "is_day_before_new_year","Season","is_start_of_month","is_end_of_month",
                           "Day_Name_sin","Day_Name_cos","Month_sin","Month_cos"]
    known_categoricals  = ["portion_type"]
    static_categoricals = [group_col, "category_str"]
    observed_reals      = [target_col,"avg_quantity_all_time","std_quantity_all_time",
                           "popularity_score","num_days_sold"]

    training = TimeSeriesDataSet(
        train_df,
        time_idx="time_idx",
        target=target_col,
        group_ids=[group_col],
        max_encoder_length=encoder_length,
        max_prediction_length=prediction_length,
        static_categoricals=static_categoricals,
        time_varying_known_reals=known_reals,
        time_varying_known_categoricals=known_categoricals,
        time_varying_unknown_reals=[target_col] + observed_reals,
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )

    # ——— הקטע החדש: דיבוג ערכים לא מוכרים ב-val_df ———
    categorical_encoders = training.get_parameters()["categorical_encoders"]
    for col, encoder in categorical_encoders.items():
        if col in val_df.columns:
            val_values = set(val_df[col].unique())
            known      = set(encoder.classes_)
            unknown    = val_values - known
            if unknown:
                print(f"⚠️ עמודה '{col}' מכילה ערכים לא מוכרים: {unknown}")
    # ——————————————————————————————————————————————

    # שלב 5 – סינון ערכים לא חוקיים לפי encoders של המודל
    before = len(val_df)
    for col, encoder in categorical_encoders.items():
        if col in val_df.columns:
            legal = encoder.classes_
            val_df = val_df[val_df[col].isin(legal)]
    after = len(val_df)
    print(f"סוננו {before - after} שורות מ-val_df עם ערכים לא חוקיים בעמודות שקודדו.")

    # שלב 6 – טיפול ידני ב-clean_desc_encoded == 54 אם נשאר
    if 54 in val_df.get("clean_desc_encoded", []):
        val_df = val_df[val_df["clean_desc_encoded"] != 54]
        print("⚠️ הסרנו ידנית את הקוד 54 מ-val_df (clean_desc_encoded).")

    # שלב 7 – יצירת validator dataset
    validation = TimeSeriesDataSet.from_dataset(training, val_df)

    return training, validation

In [9]:
training, validation = build_tft_datasets_from_raw(
    df,
    desc_map,
    cat_map,
    cutoff_date="2024-01-01",
    encoder_length=30,
    prediction_length=7)

/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:951: UserWarning: Target scales will be only added for continous targets
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:1831: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 80 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__clean_desc_str': '10 קלאסיות'}, {'__group_id__clean_desc_str': '2 פיצות ללא גלוטן משפחתיות'}, {'__group_id__clean_desc_str': '2 פיצות משפחתיות קלאסיות rl'}, {'__group_id__clean_desc_str': '3 משולשי קרטון לפיצה'}, {'__group_id__clean_desc_str': '3 פלפלים פפרוציני'}, {'__group_id__clean_desc_str': '4 פיצות 14'}, {'__group_id__clean_desc_str': '5 כדורי פירה'}, {'__group_id__clean_desc_str': '6 גבינות איטסיין'}, {'__group_id__clean_desc_str': '6 פיצות מרגריטה'},

⚠️ עמודה 'clean_desc_str' מכילה ערכים לא מוכרים: {'נקניקייה טבעונית', 'פיצה l קלאסית תוספת צמחונית משלוח po', '2 משפחתיות קלאסיות', '3 פיצות ללא גלוטן משפחתיות', '6 גבינות 14 דקה', 'עם אורגנו', 'קלפי ליגת האלופות', 'גבינת עיזים', '3 תוספות חינם', 'עם תבלין איטלקי', '6 גבינות 14', 'רולס פירות יער שמנת', 'עם רוטב פיסטוק', 'קרם שוקולד נוגט', 'רוטבים', 'רולס בייגלה שמנת ועוגיות', 'גרליק בייטס 24 יחידות', 'מונסטר אולטרה', 'פיצה מיוחדת ממהדורת חורף פיצה קלאסית', 'רולס מרשמלו שמנת ועוגיות', 'אצבעות פסטו קרונפלקס', 'קורנפלקס דליס', '30 קינוח', 'הבלקנית 14', 'עם רוטב בייגלה', 'רולס פצפוצי רושה ופיסטוק שמנת', 'עם רוטב מרשמלו', '5 גבינות', 'פיצה מיוחדת ממהדורת חורף 2 פחיות או מנת נלוות', '8 טבעות גאודה מצופות', 'קלאסית 12', 'ציפס', 'מונסטר מנגו לוקו', 'אצבעות גבינה פסח', 'עם רוטב פירות יער', 'קרונפלקס פטריות 14', 'פיצה l קלאסית מנה נלוות קינוח', '2 יח עוגיות מדלן שוקולדציפס', 'פיצה משפחתית קלאסית באיסוף בנק מזרחי', 'משפחתית קלאסית 2 פחיות', 'שקית לקוח', 'פיצה משפחתית מנה נלווית שתיה גדולה'}
סוננו

KeyError: "Unknown category '54' encountered. Set `add_nan=True` to allow unknown categories"

In [ ]:
# debug: בדוק איפה בדיוק 54 חומק
categorical_encoders = training.get_parameters()["categorical_encoders"]
for col, encoder in categorical_encoders.items():
    # רק אם העמודה הזו באמת קיימת ב־val_df
    if col in val_df.columns:
        val_values = set(val_df[col].unique())
        known = set(encoder.classes_)
        unknown = val_values - known
        if unknown:
            print(f"⚠️ עמודה '{col}' מכילה ערכים לא מוכרים: {unknown}")
